In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [5]:
from evoscaper.scripts.init_from_hpos import init_from_hpos
from evoscaper.utils.math import arrayise
from evoscaper.utils.preprocess import make_datetime_str
from evoscaper.utils.visualise import vis_sampled_histplot
from bioreaction.misc.misc import load_json_as_dict
from synbio_morpher.utils.results.analytics.timeseries import calculate_adaptation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import jax.numpy as jnp
import jax

jax.config.update('jax_platform_name', 'cpu')
jax.devices()

[CpuDevice(id=0)]

In [6]:
top_write_dir = os.path.join('data', '05_hidden_size', make_datetime_str())
os.makedirs(top_write_dir, exist_ok=True)
df_hpos = pd.DataFrame(load_json_as_dict(
    'data/2025_01_15__10_59_22/df_hpos_main.json'))
df_hpos2 = pd.DataFrame(load_json_as_dict(
    'data/2025_01_13__16_31_26/df_hpos_main.json'))
# i = (df_hpos['run_successful'] == 'TO_BE_RECORDED').index
i = (df_hpos['run_successful'].isin(['TO_BE_RECORDED', False])).index
df_hpos.loc[i] = df_hpos2.loc[i]
df_hpos['mi_mean'] = np.nan
df_hpos.loc[df_hpos['run_successful'], 'mi_mean'] = df_hpos[df_hpos['run_successful']]['mutual_information_conditionality'].apply(np.array).apply(np.mean)
(~df_hpos['run_successful']).sum()

/tmp/ipykernel_2581943/854693653.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_hpos.loc[i] = df_hpos2.loc[i]


-57

In [24]:
def vis_r2(d, save_path):
    plt.figure()
    sns.lineplot(d, x='hidden_size', y='R2_test')
    sns.scatterplot(d, x='hidden_size', y='R2_test')
    plt.title('R2 score on test set')
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()


def vis_mi(d, save_path):
    plt.figure()
    sns.scatterplot(d, x='hidden_size', y='mi_mean')
    plt.title('Mutual information between hidden z and conditional input')
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()


def load_params(fn_saves):
    saves_loaded = load_json_as_dict(fn_saves)
    params = saves_loaded[str(list(saves_loaded.keys())[-1])]['params']
    params = arrayise(params)
    return params


def get_analytics(hpos):
    analytics = load_json_as_dict(os.path.join(
        os.path.dirname(hpos['filename_saved_model']), 'analytics.json'))
    analytics['sensitivity_wrt_species-6'] = np.array(
        analytics['sensitivity_wrt_species-6'])
    analytics['precision_wrt_species-6'] = np.array(
        analytics['precision_wrt_species-6'])
    for k in ['sensitivity_wrt_species-6', 'precision_wrt_species-6']:
        # analytics[k] = np.where(np.isnan(
        #     analytics[k]), 0, analytics[k])
        analytics[f'Log {k.split("_")[0]}'] = np.log10(analytics[k])
    analytics['adaptation'] = calculate_adaptation(
        analytics['sensitivity_wrt_species-6'], analytics['precision_wrt_species-6'])

    nbin = hpos['prep_y_categorical_n_bins']
    analytics['Log sensitivity'] = analytics['Log sensitivity'].reshape(
        nbin, analytics['Log sensitivity'].shape[0]//nbin, -1)
    return analytics


def plot_bars(analytics, y_datanormaliser, idx_output, save_path):
    sampled_cond = np.array(
        list(y_datanormaliser.metadata['Log sensitivity']['category_map'].values()))
    means_s = jax.vmap(lambda x, c: jnp.nanmean(
        x[..., idx_output]) - c)(analytics['Log sensitivity'], sampled_cond)
    sns.barplot(means_s, palette='viridis')
    plt.xlabel('Condition input')
    plt.ylabel('Difference in sensitivity')
    plt.title('Difference in sensitivity of target prompt and mean actual')
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()


def vis_sampled_histplot(analytic, y_datanormaliser, idx_output: int, output_species,
                         title: str, x_label: str, multiple='fill', show=False, f=sns.histplot, save_path=None, **kwargs):
    if f == sns.histplot:
        for k, v in zip(('element', 'bins', 'log_scale'), ('step', 20, [True, False])):
            kwargs.setdefault(k, v)
    category_array = np.array(sorted(y_datanormaliser.metadata[y_datanormaliser.cols_separate[0]]["category_map"].values())).repeat(
        len(analytic)//len(y_datanormaliser.metadata[y_datanormaliser.cols_separate[0]]["category_map"]))

    fig = plt.figure(figsize=(13, 4))
    fig.subplots_adjust(wspace=0.6)
    for i, output_specie in enumerate(output_species):
        title_curr = title + f': species {output_specie}'
        df_s = pd.DataFrame(columns=[x_label, 'VAE conditional input'],
                            data=np.concatenate([analytic[:, idx_output][:, None], category_array[:, None]], axis=-1))
        df_s['VAE conditional input'] = df_s['VAE conditional input'].astype(
            float).apply(lambda x: f'{x:.2f}')
        ax = plt.subplot(1, 2, i+1)
        f(df_s, x=x_label,
          multiple=multiple, hue='VAE conditional input', palette='viridis',
          **kwargs)

        sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        plt.title(title_curr)

    if show:
        plt.show()

    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()


def vis_bars_all(d, idx_output, top_write_dir):
    d = d[[os.path.exists(os.path.join(os.path.dirname(hpos['filename_saved_model']), 'analytics.json')) for i, hpos in d.iterrows()]]
    all_c = []
    col_c = d.iloc[0]['objective_col'][0]
    for i, hpos in d.iterrows():
        analytics = get_analytics(hpos)
        all_c.append(analytics[col_c])
    all_c = np.array(all_c)

    (
        rng, rng_model, rng_dataset,
        config_norm_x, config_norm_y, config_filter, config_optimisation, config_dataset, config_training, config_model,
        data, x_cols, df,
        x, cond, y, x_train, cond_train, y_train, x_val, cond_val, y_val,
        total_ds, n_batches, BATCH_SIZE, x_datanormaliser, x_methods_preprocessing, y_datanormaliser, y_methods_preprocessing,
        _,
        encoder, decoder, model, h2mu, h2logvar, reparam
    ) = init_from_hpos(hpos)
    sampled_cond = np.array(
        list(y_datanormaliser.metadata[col_c]['category_map'].values()))

    def get_means(c):
        return jax.vmap(lambda x, cc: jnp.nanmean(
            x[..., idx_output]) - cc)(c, sampled_cond)
    c_means = jax.vmap(get_means)(all_c[..., idx_output])
    
    plt.plot([0, len(c_means)], [0, 0], '--')
    sns.violinplot(c_means.T, palette='viridis')
    plt.xticks(d['hidden_size'].to_numpy())
    plt.savefig(os.path.join(top_write_dir, 'violins_all.png'), bbox_inches='tight', dpi=300)
    plt.close()
    
    sns.barplot(np.abs(c_means.T), palette='viridis')
    plt.xticks(d['hidden_size'].to_numpy())
    plt.savefig(os.path.join(top_write_dir, 'bars_all.png'), bbox_inches='tight', dpi=300)
    plt.close()



def run_hpos(hpos, dir_save):
    (
        rng, rng_model, rng_dataset,
        config_norm_x, config_norm_y, config_filter, config_optimisation, config_dataset, config_training, config_model,
        data, x_cols, df,
        x, cond, y, x_train, cond_train, y_train, x_val, cond_val, y_val,
        total_ds, n_batches, BATCH_SIZE, x_datanormaliser, x_methods_preprocessing, y_datanormaliser, y_methods_preprocessing,
        _,
        encoder, decoder, model, h2mu, h2logvar, reparam
    ) = init_from_hpos(hpos)

    params = load_params(hpos['filename_saved_model'])

    h_all = encoder(params, rng, np.concatenate([x, cond], axis=-1))
    h_all = h_all.reshape(np.prod(h_all.shape[:-1]), -1)
    cond_rev_all = y_datanormaliser.create_chain_preprocessor_inverse(y_methods_preprocessing)(
        cond, col=config_dataset.objective_col[0]).reshape(np.prod(cond.shape[:-1]), -1).squeeze()
    x_rev_all = x_datanormaliser.create_chain_preprocessor_inverse(
        x_methods_preprocessing)(x).reshape(np.prod(x.shape[:-1]), -1).squeeze()

    idx_output = -1
    analytics = get_analytics(hpos)
    plot_bars(analytics, y_datanormaliser, idx_output,
              os.path.join(dir_save, 'bars.png'))

    vis_sampled_histplot(analytics['sensitivity_wrt_species-6'], y_datanormaliser, idx_output, config_dataset.output_species,
                         title=f'Sensitivity of generated circuits', x_label=f'Log10 of sensitivity to signal {config_dataset.signal_species[0]}', multiple='layer', show=False,
                         f=sns.kdeplot, log_scale=[True, False], fill=False, save_path=os.path.join(dir_save, f'kde.png'))

In [25]:
d = df_hpos[df_hpos['run_successful']]
idx_output = -1
vis_bars_all(d, idx_output, top_write_dir)


In [15]:
d = df_hpos[df_hpos['run_successful']]
vis_r2(d, save_path=os.path.join(top_write_dir, 'r2.png'))
vis_mi(d, save_path=os.path.join(top_write_dir, 'mi.png'))
idx_output = -1
vis_bars_all(d, idx_output, save_path=os.path.join(top_write_dir, 'bars_all.png'))

for i, hpos in df_hpos.iterrows():
    if not hpos['run_successful'] or not(os.path.exists(os.path.join(os.path.dirname(hpos['filename_saved_model']), 'analytics.json'))):
        continue
    
    os.makedirs(os.path.join(top_write_dir, str(i)), exist_ok=True)
    run_hpos(hpos, dir_save=os.path.join(top_write_dir, str(i)))

/tmp/ipykernel_2490922/2698233196.py:49: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(means_s, palette='viridis')
/tmp/ipykernel_2490922/2698233196.py:49: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(means_s, palette='viridis')
/tmp/ipykernel_2490922/2698233196.py:49: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(means_s, palette='viridis')
/tmp/ipykernel_2490922/2698233196.py:49: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` 